# Input

I researched and created this proof of concept for n-fold cross validation for us to implement in our experiment. 

In [7]:
# Importing libraries 
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error 

from numpy import mean
from numpy import std

In [8]:
# Create df 
df = pd.read_csv('/datc/nano/notebooks/Target variable & Features (V3).csv', index_col = 0)
df_yen = df[df['Threshold method'] =='yen']
df_yen.head()

x = df_yen[['Threshold: separation', 'Threshold: intensity', 'Threshold: fill', 'Threshold: count', 'Threshold: border', 'Threshold: area spread']]
y = df_yen[['User score']]

# k-fold cross validation

Source: https://machinelearningmastery.com/k-fold-cross-validation/

Any preparation of the data prior to fitting the model occur on the CV-assigned training dataset within the loop rather than on the broader data set. This also applies to any tuning of hyperparameters. 

How many folds are a good fit?
- k=10: 10 folds has been found through experimentation to generally result in a model skill estimate with low bias a modest variance.
- k=n: Number of experiments with the dataset, so that each validation set gets used for one experiment.

In [10]:
# Cross validation
kf = KFold(n_splits=3, random_state=42, shuffle=True)

kf_mae = []
kf_scores = []

for train_ind, valid_ind in kf.split(x):
    train_x = x.iloc[train_ind].to_numpy()
    valid_x = x.iloc[valid_ind].to_numpy()
    train_y = y.iloc[train_ind].to_numpy()
    valid_y = y.iloc[valid_ind].to_numpy()
    
    # Model
    model = LogisticRegression(multi_class='auto', random_state=0, max_iter=5000, class_weight='balanced')
    model.fit(train_x, train_y.ravel())
    y_pred = model.predict(valid_x)
   
    # MAE
    mse = mean_absolute_error(valid_y, y_pred)
    kf_mae.append((mse))
    
    # Scores
    scores = cross_val_score(model, train_x, train_y, scoring='accuracy', cv=kf, n_jobs=-1)
    kf_scores.append(mean(scores))

# Mean MAE
mae_mean = sum(kf_mae)/len(kf_mae)
print(mae_mean)

# Mean cross-val-score
scores_mean = sum(kf_scores)/len(kf_scores)
print(scores_mean)

3.390350877192983
0.10327635327635327


## Output

The research on the number of folds was considered for our experiment, but not chosen finally. Instead, we went with 5 folds.